<a href="https://colab.research.google.com/github/Clustering-Crew/UNIV-6080-Notebooks/blob/main/SingularValueDecomposition_With_Exercise_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Singular Value Decomposition

Let's work through Example 4.13 using NumPy.

We would like to compute the singular value decomposition (SVD) of $\mathbf{A} =
\begin{bmatrix}
1  & 0 & 1\\
-2 & 1 & 0\\
\end{bmatrix}
$.

Step 1 is to compute the right-singular vectors as the eigenbasis of $\mathbf{A}^{\top}\mathbf{A}$:

In [2]:
A = np.array([[1., 0., 1.], [-2., 1., 0.]])
d, V = np.linalg.eig(np.dot(A.T, A))
print(d)
print(V)

[ 6.0000000e+00 -4.4408921e-16  1.0000000e+00]
[[ 0.91287093 -0.40824829  0.        ]
 [-0.36514837 -0.81649658  0.4472136 ]
 [ 0.18257419  0.40824829  0.89442719]]


Note that `numpy.linalg.eig` does not necessarily order its eigenvalues. By convention, we will sort the eigenvectors from largest to smallest eigenvalue:

In [3]:
idx = d.argsort()[::-1]
d = d[idx]
V = V[:, idx]
print(d)
print(V)

[ 6.0000000e+00  1.0000000e+00 -4.4408921e-16]
[[ 0.91287093  0.         -0.40824829]
 [-0.36514837  0.4472136  -0.81649658]
 [ 0.18257419  0.89442719  0.40824829]]


Step 2 is to compute the singular-value matrix. The singular values are the square roots of the eigenvalues of $\mathbf{A}^{\top}\mathbf{A}$. Since the rank of $\mathbf{A}=2$, there are only two non-zero singular values. Remember that the singular value matrix must be the same size as $\mathbf{A}$, so we pad with zeros accordingly:

In [4]:
min_dim = min(A.shape)
nz_idx = ~np.isclose(d, np.zeros_like(d))  # index to non-zero elements
Sigma = np.zeros_like(A)
# This next step is careful to construct Sigma with the right zero padding
Sigma[:min_dim, :min_dim] = np.diag(np.sqrt(d[nz_idx]))
print(Sigma)

[[2.44948974 0.         0.        ]
 [0.         1.         0.        ]]


We see that indeed $\mathbf{\Sigma}=
\begin{bmatrix}
\sqrt{6} & 0 & 0\\
0        & 1 & 0
\end{bmatrix}$.

Step 3 is to compute the left-singular vectors as the normalized image of the right-singular vectors:

In [5]:
m = A.shape[0]
U = np.zeros((m, m))
for i in range(m):
  U[:, i] = (1/Sigma.flat[::Sigma.shape[1]+1][i]) * np.dot(A, V[:, i])
print(U)

[[ 0.4472136   0.89442719]
 [-0.89442719  0.4472136 ]]


We see that indeed $\mathbf{U} =
\frac{1}{\sqrt{5}}
\begin{bmatrix}
1  & 2\\
-2 & 1
\end{bmatrix}.$

Note that this procedure is numerically unstable, and we can typically compute the SVD without resorting to the eigenvalue decomposition of $\mathbf{A}^{\top}\mathbf{A}$. `numpy.linalg.svd` provides a convenient way to do this:

In [6]:
u, s, vh = np.linalg.svd(A)  # using the variable names in numpy docs
print(u)
print(s)
# The "h" in vh refers to the Hermetian, which is the complex generalization
# of the transpose
# Effectively, this is V.T
print(vh)

[[-0.4472136   0.89442719]
 [ 0.89442719  0.4472136 ]]
[2.44948974 1.        ]
[[-9.12870929e-01  3.65148372e-01 -1.82574186e-01]
 [-3.73536832e-16  4.47213595e-01  8.94427191e-01]
 [-4.08248290e-01 -8.16496581e-01  4.08248290e-01]]


You will notice, that for the exception of negative signs that appear in $\mathbf{U}$ and $\mathbf{V}^{\top}$ and cancel out, that this is the same decomposition that we computed above. We can verify that it implements the linear mapping $\mathbf{A}$:

In [7]:
smat = np.zeros_like(A)
smat[:min_dim, :min_dim] = np.diag(s)
np.allclose(u @ smat @ vh, A)  # note that @ is matrix multiplication

True

In [8]:
print(u.shape)
print(smat.shape)
print(vh.shape)


(2, 2)
(2, 3)
(3, 3)


In [9]:
b = u @ smat
print(b)

[[-1.09544512  0.89442719  0.        ]
 [ 2.19089023  0.4472136   0.        ]]


## Exercise

Verify that the SVD implements $\mathbf{A}$ like we did above, but without having to explicitly compute `smat`. In other words, leave `s` in vector form. Hint: what part of `vh` is not used at all in `u @ smat @ vh`?

In [10]:
'''Using the hint, we understand that since the third column of smat will be all zeroes,
the third row of vh is not needed. '''

u, s, vh = np.linalg.svd(A, full_matrices=True)

b = (u * s) @ vh[:len(s), :] # Scale matrix u with the singular values.

np.allclose(b, A)

True